In [1]:
import cirq
from math import pi
import numpy as np

In [2]:
#pair generation circuit class

class pair_generation(cirq.Gate):
    def __init__(self):
        super (pair_generation, self)
        
    def _num_qubits_(self):
        return 4
    
    def _circuit_(self, qubits, theta):
        q0, q1, q2, q3 = qubits
        theta = np.pi/8
        
        yield cirq.CX(q1, q0)
        yield cirq.CX(q3, q1)
        yield cirq.CZ(q0, q1)
        yield cirq.H(q0)
        yield cirq.CX(q3, q2)
        yield cirq.ry(rads= theta).on(q2)
        yield cirq.ry(rads=- theta).on(q3)
        yield cirq.CX(q0, q3)
        yield cirq.CX(q0, q2)
        yield cirq.ry(rads=- theta).on(q3)
        yield cirq.ry(rads= theta).on(q2)
        yield cirq.CX(q1, q2)
        yield cirq.CX(q2, q3)
        yield cirq.ry(rads=- theta).on(q2)
        yield cirq.ry(rads= theta).on(q3)
        yield cirq.CX(q0, q3)
        yield cirq.CZ(q0, q3)
        yield cirq.ry(rads=- theta).on(q2)
        yield cirq.ry(rads= theta).on(q3)
        yield cirq.CX(q3, q2)
        yield cirq.CX(q1, q3)
        yield cirq.H(q3)
        yield cirq.CX(q3, q1)
        yield cirq.CX(q1, q0)
        
    def _circuit_diagram_info_(self, args):
        return ["Px"]*self._num_qubits_()

        

In [12]:
# single circuit testing
angle = np.pi / 8
circuit= cirq.Circuit()
q0, q1, q2, q3= cirq.LineQubit.range(4)

circuit.append(pair_generation().on(q0,q1,q2,q3))
print(circuit)

0: ───Px───
      │
1: ───Px───
      │
2: ───Px───
      │
3: ───Px───


In [3]:
#given rotation generation

class SqrtISWAP(cirq.Gate):
    def __init__(self):
        super(SqrtISWAP, self)

    def _num_qubits_(self):
        return 2
    
    # Obtained from: https://github.com/quantumlib/Cirq/blob/a22269dfe41b0da78243bbd210a915d26cc7d25f/cirq-core/cirq/ops/swap_gates.py#L166
    def _decompose_(self, qubits):
        q0, q1 = qubits
        yield cirq.CX(q0,q1)
        yield cirq.H(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=0.25).on(q0)
        yield cirq.CX(q1,q0)
        yield cirq.ZPowGate(exponent=-0.25).on(q0)
        yield cirq.H(q0)
        yield cirq.CX(q0,q1)
        
    def _circuit_diagram_info_(self, args):
        return ["sqrt(iSWAP)"] * self.num_qubits()

In [4]:
class IonSupportedGivens(cirq.Gate):
    def __init__(self, theta):
        super(IonSupportedGivens, self)
        self.theta = theta

    def _num_qubits_(self):
        return 2

    def _decompose_(self, qubits):
        
        # Have to EXPLICITLY call decomposition, Azure Quantum doesn't seem to auto-decompose
        # to IonQ supported gates
        q0, q1 = qubits
        yield SqrtISWAP().on(q0,q1)._decompose_()
        
        yield cirq.Rz(rads= -self.theta).on(q0)
        yield cirq.Rz(rads=self.theta + np.pi).on(q1)

        yield SqrtISWAP().on(q0,q1)._decompose_()
        yield cirq.Rz(rads=np.pi).on(q1)
        
    def _circuit_diagram_info_(self, args):
        return ["IonGivens({:.3f})".format(self.theta)] * self.num_qubits()

In [5]:
#make a circuit with a random angle to test

#https://pubs.rsc.org/en/content/articlelanding/2022/sc/d1sc05691c#cit39
#from fig 1
class vqe(cirq.Gate):
    def __init__(self, theta):
        super(vqe, self)
        self.theta = theta
        
    def _num_qubits_(self):
        return 8
    
    def _vqe_circuit_(self, qubits):
        q0, q1, q2, q3, q4, q5, q6, q7 = qubits
        
        yield IonSupportedGivens(theta = self.theta).on(q0, q2)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q4, q6)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q1, q3)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q5, q7)._decompose_()
        yield IonSupportedGivend(theta = self.theta).on(q2, q4)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q3, q5)._decoompose_()
        yield IonSupportedGivens(theta = self.theta).on(q0, q2)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q4, q6)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q1, q3)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q5, q7)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q2, q4)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q3, q5)._decompose_()
        yield pair_generation().on(q0, q1, q2, q3)._circuit_()
        yield pair_generation().on(q4, q5, q6, q7)._circuit_()
        yield pair_generation().on(q2, q3, q4, q5)._circuit_()
        
        yield pair_generation().on(q0, q1, q2, q3)._circuit_()
        yield pair_generation().on(q4, q5, q6, q7)._circuit_()
        yield pair_generation().on(q2, q3, q4, q5)._circuit_()
        yield IonSupportedGivens(theta = self.theta).on(q0, q2)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q4, q6)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q1, q3)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q5, q7)._decompose_()
        yield IonSupportedGivend(theta = self.theta).on(q2, q4)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q3, q5)._decoompose_()
        yield IonSupportedGivens(theta = self.theta).on(q0, q2)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q4, q6)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q1, q3)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q5, q7)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q2, q4)._decompose_()
        yield IonSupportedGivens(theta = self.theta).on(q3, q5)._decompose_()
        
        
    def _circuit_diagram_info_(self, args):
        return ["VQE".format(self.theta)] * self.num_qubits()
        

In [6]:
# test circuti
angle = np.pi/8

vqe_cir = cirq.Circuit()
q0, q1, q2, q3, q4, q5, q6, q7 = cirq.LineQubit.range(8)

vqe_cir.append(vqe(theta = angle).on(q0, q1, q2, q3, q4, q5, q6, q7))
print(vqe_cir)

0: ───VQE───
      │
1: ───VQE───
      │
2: ───VQE───
      │
3: ───VQE───
      │
4: ───VQE───
      │
5: ───VQE───
      │
6: ───VQE───
      │
7: ───VQE───
